### Create Spark Session

In [271]:
import os
from pyspark.sql import SparkSession
import warnings

warnings.filterwarnings('ignore')

spark = (
    SparkSession
        .builder
        .config(
            "spark.driver.host", 
            "localhost"
        )
        .config(
            "spark.jars",
            "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
        .config(
            "spark.driver.extraClassPath",
            "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
        .appName('appname')
        .getOrCreate()
)

spark.conf.set("spark.sql.ansi.enabled", False)

print("Spark session created succesfully...")

Spark session created succesfully...


### Import Libraries

In [272]:
import sqlite3
import pyspark.pandas as ps
import os
from datetime import date, datetime

print("Libraries imported successfully...")

Libraries imported successfully...


### Global variables

In [273]:
db = "wwi.db"
db_wh = "wwi_wh.db"
url_db = "jdbc:sqlite:{}/wwi.db".format(os.getcwd())
url_db_wh = "jdbc:sqlite:{}/wwi_wh.db".format(os.getcwd())
new_cutoff_date = date(2013, 1, 1)
last_cutoff_date = date(2013, 1, 1)
initial_load_date = date(2013, 1, 1)
new_cutoff = new_cutoff_date.strftime("%Y-%m-%d")
last_cutoff = last_cutoff_date.strftime("%Y-%m-%d")
initial_load = initial_load_date.strftime("%Y-%m-%d")

print("Global variables loaded...")

Global variables loaded...


### Functions

Function to execute multiple sql scripts

In [274]:
def sql_execute_scripts(sql_scripts, destination_url):
    con = sqlite3.connect(destination_url)
    cur = con.cursor()

    try:
        for sql_script in sql_scripts:
            cur.execute(sql_script)

        # Save (commit) the changes
        con.commit()
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        raise e
    finally:
        con.close()    

print("Function sql_execute_scripts loaded...")

Function sql_execute_scripts loaded...


Function to insert load history entry

In [275]:
def insert_load_history_entry(table_name, load_date):
    # retrieve data
    df = ps.read_sql("""
        SELECT 
            *
        FROM 
            LoadHistory lh 
        WHERE 
            TableName = '{{table_name}}' AND
            DATETIME(LoadDate) <= DATETIME('{{load_date}}') AND 
            Status = 'Successful'
        ORDER BY 
            LoadDate DESC
        LIMIT 1 
        """.replace(
            "{{table_name}}", 
            table_name
        ).replace(
            "{{load_date}}", 
            load_date
        ),
        con=url_db_wh
    )

    if len(df) > 0:
        last_cutoff = df["LoadDate"]

    sql_execute_scripts(
        [
            """
                INSERT INTO LoadHistory 
                (
                    TableName,
                    LoadDate,
                    Status
                )
                VALUES
                (
                    '{{table_name}}',
                    '{{new_cutoff}}',
                    NULL
                )
            """.replace(
                "{{table_name}}", 
                table_name
            ).replace(
                "{{new_cutoff}}", 
                load_date
            )
        ],
        db_wh
    )

print("Function insert_load_history_entry loaded...")

Function insert_load_history_entry loaded...


Function to update

In [276]:
def update_load_history_entry_status(table_name, load_date, status):
    sql_execute_scripts(
        [
            """
                UPDATE 
                    LoadHistory 
                SET
                    Status = '{{status}}'
                WHERE 
                    TableName = '{{table_name}}' AND 
                    DATETIME(LoadDate) = DATETIME('{{new_cutoff}}')
            """.replace(
                "{{table_name}}", 
                table_name
            ).replace(
                "{{new_cutoff}}", 
                load_date
            ).replace(
                "{{status}}",
                status
            )
        ],
        db_wh
    )

print("Function update_load_history_entry_status loaded...")

Function update_load_history_entry_status loaded...


Function to build columns

In [277]:
def get_columns_str(columns):
    sql_columns = "" 
    
    for column in columns:
        sql_columns += column + ", \n" 
    sql_columns = sql_columns[:-3]

    return sql_columns

print("Function get_columns_str loaded...")

Function get_columns_str loaded...


Function to save changed table

In [278]:
def save_changed_table(columns, table_name, archive_table_name, save_table_name, url, additional_where=""):
    if table_exists(save_table_name + "_" + new_cutoff, url):
        print("Table exists " + save_table_name + "_" + new_cutoff)
        return
    
    sql = """
        SELECT
            {{sql_columns}}
        FROM
            {{sql_table_name}}
        WHERE
            (
                DATETIME(ValidFrom) > DATETIME('{{last_cutoff}}') OR
                (
                    DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                    DATETIME(ValidFrom) >= DATETIME('{{last_cutoff}}')
                )
            ) AND
            (
                DATETIME('{{new_cutoff}}') <= DATETIME(ValidTo) OR
                DATETIME(ValidTo) IS NULL 
            ) AND
            DATETIME(ValidFrom) <= DATETIME('{{new_cutoff}}')
            {{sql_additional_where}}

        UNION

        SELECT
            {{sql_columns}}
        FROM
            {{sql_archive_table_name}}
        WHERE
            (
                DATETIME(ValidFrom) > DATETIME('{{last_cutoff}}') OR
                (
                    DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                    DATETIME(ValidFrom) >= DATETIME('{{last_cutoff}}')
                )
            ) AND
            (
                DATETIME('{{new_cutoff}}') <= DATETIME(ValidTo) OR
                DATETIME(ValidTo) IS NULL 
            ) AND
            DATETIME(ValidFrom) <= DATETIME('{{new_cutoff}}')
            {{sql_additional_where}}
    """

    # Set sql columns
    sql_columns = get_columns_str(columns)

    # replace placeholders accordingly
    sql = sql.replace("{{sql_columns}}", sql_columns)
    sql = sql.replace("{{sql_table_name}}", table_name)
    sql = sql.replace("{{sql_archive_table_name}}", archive_table_name)
    sql = sql.replace("{{last_cutoff}}", last_cutoff)
    sql = sql.replace("{{new_cutoff}}", new_cutoff)
    sql = sql.replace("{{initial_load}}", initial_load)
    sql = sql.replace("{{sql_additional_where}}", additional_where)

    # retrieve data
    df = ps.read_sql(
        sql,
        con=url
    )

    # saved changed data
    df.spark.to_spark_io(
        format="jdbc",
        mode="overwrite",
        dbtable="[" + save_table_name + "_" + new_cutoff + "]",
        url=url
    )

    print("Successfully created " + save_table_name + " table.")

print("Function save_changed_table loaded...")

Function save_changed_table loaded...


Function to save temp table

In [279]:
def save_temp_table(sql_template, sql_archive_where, table_name, archive_table_name, save_table_name, source_url, destination_url):
    sql = sql_template.replace("{{sql_table_name}}", table_name)
    sql = sql.replace("{{sql_archive_where}}", "")

    df = ps.read_sql(
        sql,
        con=source_url
    )

    df.spark.to_spark_io(
        format="jdbc",
        mode="overwrite",
        dbtable=save_table_name,
        url=source_url
    )

    if len(archive_table_name) > 0:
        sql = sql_template.replace("{{sql_table_name}}", archive_table_name)
        sql = sql.replace("{{sql_archive_where}}", sql_archive_where)
        df = ps.read_sql(
            sql,
            con=source_url
        )
        df.spark.to_spark_io(
            format="jdbc",
            mode="append",
            dbtable=save_table_name,
            url=source_url
        )

    df = ps.read_sql(
        "SELECT * FROM " + save_table_name,
        con=source_url
    )
    df.spark.to_spark_io(
        format="jdbc",
        mode="overwrite",
        dbtable=save_table_name,
        url=destination_url
    )

    print("Successfully created " + save_table_name + " table.")

print("Function save_temp_table loaded...")

Function save_temp_table loaded...


Function to save available table

In [280]:
def save_available_table(columns, table_name, archive_table_name, save_table_name, url):
    if table_exists(save_table_name + "_" + new_cutoff, url):
        print("Table exists " + save_table_name + "_" + new_cutoff)
        return

    sql = """
    
        SELECT
            {{sql_columns}}
        FROM
            {{sql_table_name}}
        WHERE
            DATETIME('{{new_cutoff}}') >= DATETIME(ValidFrom) AND 
            (
                DATETIME('{{new_cutoff}}') <= DATETIME(ValidTo) OR 
                DATETIME(ValidTo) IS NULL
            )

        UNION

        SELECT
            {{sql_columns}}
        FROM
            {{sql_archive_table_name}}
        WHERE
            DATETIME('{{new_cutoff}}') >= DATETIME(ValidFrom) AND 
            (
                DATETIME('{{new_cutoff}}') <= DATETIME(ValidTo) OR 
                DATETIME(ValidTo) IS NULL
            )
    """

    # Set sql columns
    sql_columns = get_columns_str(columns) 

    # replace placeholders accordingly
    sql = sql.replace("{{sql_columns}}", sql_columns)
    sql = sql.replace("{{sql_table_name}}", table_name)
    sql = sql.replace("{{sql_archive_table_name}}", archive_table_name)
    sql = sql.replace("{{new_cutoff}}", new_cutoff)

    # retrieve data
    df = ps.read_sql(
        sql,
        con=url
    )

    # saved changed data
    df.spark.to_spark_io(
        format="jdbc",
        mode="overwrite",
        dbtable="[" + save_table_name + "_" + new_cutoff + "]",
        url=url
    )

    print("Successfully created " + save_table_name + " table.")

print("Function save_available_table loaded...")

Function save_available_table loaded...


Function to check it table exists

In [281]:
def table_exists(table_name, url):
    df = ps.read_sql("""
        SELECT 
            name
        FROM 
            sqlite_master 
        WHERE 
            type='table' AND 
            name='{{table_name}}'
        """.replace("{{table_name}}", table_name),
        con=url
    )

    return len(df) > 0

print("Function table_exists loaded...")

Function table_exists loaded...


Function to check if table is already populated for the new cutoff

In [282]:
def table_loaded_for_date(table_name, load_date):
    df = ps.read_sql("""
        SELECT
            *
        FROM
            LoadHistory
        WHERE
            TableName = '{{table_name}}' AND
            DATETIME(LoadDate) >= DATETIME('{{load_date}}') AND
            Status = 'Successful'
        """.replace(
            "{{table_name}}", 
            table_name
        ).replace(
            "{{load_date}}",
            load_date
        ),
        con=url_db_wh
    )

    return len(df) > 0

print("Function table_loaded_for_date loaded...")

Function table_loaded_for_date loaded...


Function to load changed tables

In [283]:
def load_changed_table(table_name):
    # Countries
    if table_name == "Countries":
        save_changed_table(
            {
                "CountryID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "Countries",
            "Countries_Archive",
            "__Countries_Changed",
            url_db
        )

    # State Provinces
    if table_name == "StateProvinces":
        save_changed_table(
            {
                "StateProvinceID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "StateProvinces", 
            "StateProvinces_Archive", 
            "__StateProvinces_Changed",
            url_db
        )

    # Customer Categories
    if table_name == "CustomerCategories":
        save_changed_table(
            {
                "CustomerCategoryID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "CustomerCategories",
            "CustomerCategories_Archive",
            "__CustomerCategories_Changed",
            url_db
        )

    # Buying Groups
    if table_name == "BuyingGroups":
        save_changed_table(
            {
                "BuyingGroupID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "BuyingGroups", 
            "BuyingGroups_Archive", 
            "__BuyingGroups_Changed",
            url_db
        )

    
    # Primary Contacts
    if table_name == "PrimaryContacts":
        save_changed_table(
            {
                "PersonID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "People", 
            "People_Archive",
            "__PrimaryContacts_Changed",
            url_db
        )

    # Employees
    if table_name == "Employees":
        save_changed_table(
            {
                "PersonID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "People",
            "People_Archive",
            "__Employees_Changed",
            url_db,
            "AND IsEmployee = 1"
        )

    # Payment Methods
    if table_name == "PaymentMethods":
        save_changed_table(
            {
                "PaymentMethodID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "PaymentMethods",
            "PaymentMethods_Archive",
            "__PaymentMethods_Changed",
            url_db
        )

    # Package Types
    if table_name == "PackageTypes":
        save_changed_table(
            {
                "PackageTypeID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "PackageTypes",
            "PackageTypes_Archive",
            "__PackageTypes_Changed",
            url_db
        )

    # Colors
    if table_name == "Colors":
        save_changed_table(
            {
                "ColorID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "Colors",
            "Colors_Archive",
            "__Colors_Changed",
            url_db
        )
    
    # Suppliers
    if table_name == "Suppliers":
        save_changed_table(
            {
                "SupplierCategoryID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "Suppliers",
            "Suppliers_Archive",
            "__SupplierCategories_Changed",
            url_db
        )

    # Transaction Types
    if table_name == "TransactionTypes":
        save_changed_table(
            {
                "TransactionTypeID", 
                "ValidFrom", 
                "ValidTo"
            }, 
            "TransactionTypes",
            "TransactionTypes_Archive",
            "__TransactionTypes_Changed",
            url_db
        )

print("Function load_changed_table loaded...")

Function load_changed_table loaded...


In [284]:
def load_available_table(table_name):
    # Countries
    if table_name == "Countries":
        save_available_table(
            {
                "CountryID",
                "CountryName",
                "Continent",
                "Region",
                "Subregion"
            }, 
            "Countries",
            "Countries_Archive",
            "__Countries_Available",
            url_db
        )

    # State Provinces
    if table_name == "StateProvinces":
        save_available_table(
            {
                "StateProvinceID",
                "CountryID",
                "StateProvinceName",
                "SalesTerritory"
            }, 
            "StateProvinces", 
            "StateProvinces_Archive", 
            "__StateProvinces_Available",
            url_db
        )

    # Customer Categories
    if table_name == "CustomerCategories":
        save_available_table(
            {
                "CustomerCategoryID",
                "CustomerCategoryName"
            }, 
            "CustomerCategories",
            "CustomerCategories_Archive",
            "__CustomerCategories_Available",
            url_db
        )

    # Buying Groups
    if table_name == "BuyingGroups":
        save_available_table(
            {
                "BuyingGroupID",
                "BuyingGroupName"
            }, 
            "BuyingGroups", 
            "BuyingGroups_Archive", 
            "__BuyingGroups_Available",
            url_db
        )
    
    # Primary Contacts
    if table_name == "PrimaryContacts":
        save_available_table(
            {
                "PersonID",
                "FullName"
            }, 
            "People", 
            "People_Archive", 
            "__PrimaryContacts_Available",
            url_db
        )

    # Customers
    if table_name == "Customers":
        save_available_table(
            {
                "CustomerID",
                "CustomerName",
                "DeliveryCityID"
            }, 
            "Customers", 
            "Customers_Archive", 
            "__Customers_Available",
            url_db
        )

    # Package Types
    if table_name == "PackageTypes":
        save_available_table(
            {
                "PackageTypeID",
                "PackageTypeName"
            }, 
            "PackageTypes",
            "PackageTypes_Archive",
            "__PackageTypes_Available",
            url_db
        )

    # Colors
    if table_name == "Colors":
        save_available_table(
            {
                "ColorID",
                "ColorName"
            }, 
            "Colors",
            "Colors_Archive",
            "__Colors_Available",
            url_db
        )

    # Supplier Categories
    if table_name == "SupplierCategories":
        save_available_table(
            {
                "SupplierCategoryID",
                "SupplierCategoryName"
            }, 
            "SupplierCategories",
            "SupplierCategories_Archive",
            "__SupplierCategories_Available",
            url_db
        )

    # Stock Items
    if table_name == "StockItems":
        save_available_table(
            {
                "StockItemID",
                "StockItemName",
                "QuantityPerOuter",
                "IsChillerStock"
            }, 
            "StockItems",
            "StockItems_Archive",
            "__StockItems_Available",
            url_db
        )

    # Transaction Types
    if table_name == "TransactionTypes":
        save_available_table(
            {
                "TransactionTypeID", 
                "TransactionTypeName"
            }, 
            "TransactionTypes",
            "TransactionTypes_Archive",
            "__TransactionTypes_Available",
            url_db
        )

print("Function load_available_table loaded...")

Function load_available_table loaded...


### Create the LoadHistory table

In [285]:
sql_execute_scripts(
    [
        """
            CREATE TABLE IF NOT EXISTS LoadHistory (
                TableName TEXT,
                LoadDate TEXT,
                Status TEXT
            )
        """
    ],
    db_wh
)

print("LoadHistory table created successfully.")

LoadHistory table created successfully.


### Populate DimDate table

In [286]:
from pyspark.sql import Row
from datetime import datetime, timedelta

table_name = "DimDate"

def get_month_name(dte, is_short):
    month_name = ""

    if dte.month == 1:
        month_name = "January"
    elif dte.month == 2:
        month_name = "February"
    elif dte.month == 3:
        month_name = "March"
    elif dte.month == 4:
        month_name = "April"
    elif dte.month == 5:
        month_name = "May"
    elif dte.month == 6:
        month_name = "June"
    elif dte.month == 7:
        month_name = "July"
    elif dte.month == 8:
        month_name = "August"
    elif dte.month == 9:
        month_name = "September"
    elif dte.month == 10:
        month_name = "October"
    elif dte.month == 11:
        month_name = "November"
    else:
        month_name = "December"

    if is_short:
        month_name = month_name[0:3]

    return month_name

def get_dimdate_fiscalmonthnumber(dte):
    if dte.month == 11 | dte.month == 12:
        return dte.month - 10
    else:
        return dte.month + 2

def get_dimdate_fiscalyear(dte):
    if dte.month == 11 | dte.month == 12:
        return dte.year + 1
    else:
        return dte.year

def get_dimdate_fiscalmonthlabel(dte):
    if dte.month == 11 | dte.month == 12:
        return "FY" + str(get_dimdate_fiscalyear(dte)) + "-" + get_month_name(dte, True)
    else:
        return "FY" + str(get_dimdate_fiscalyear(dte)) + "-" + get_month_name(dte, True)

def get_dimdate_row(dte):
    return Row(
        Date=dte, 
        DayNumber=dte.day,
        Day=dte.day,
        Month=get_month_name(dte, False),
        ShortMonth=get_month_name(dte, True),
        CalendarMonthNumber=dte.month,
        CalendarMonthLabel="CY" + str(dte.year) + "-" + get_month_name(dte, True),
        CalendarYear=dte.year,
        CalendarYearLabel="CY" + str(dte.year),
        FiscalMonthNumber=get_dimdate_fiscalmonthnumber(dte),
        FiscalMonthLabel=get_dimdate_fiscalmonthlabel(dte),
        FiscalYear="FY" + str(get_dimdate_fiscalyear(dte))
    )

def populate_dimdate():
    initial_year = initial_load_date.year
    current_year = new_cutoff_date.year
    
    for year in range(initial_year, current_year + 1):
        df = []
        first_year_date = date(year, 1, 1)

        if table_loaded_for_date(table_name, first_year_date.strftime("%Y-%m-%d")):
            print(table_name + " is already loaded for " + first_year_date.strftime("%Y-%m-%d") + ".")
        else:
            insert_load_history_entry(table_name, first_year_date.strftime("%Y-%m-%d"))

            for i in range(0, 366): 
                current_date = first_year_date + timedelta(days=i)

                if year == current_date.year:
                    df.append(get_dimdate_row(current_date))

            if len(df) > 0:
                df = spark.createDataFrame(df)
                df = df.toPandas()

                conn = sqlite3.connect(db_wh)
                df.to_sql(
                    table_name, 
                    conn, 
                    if_exists='append', 
                    index=False
                )
                conn.close()

            update_load_history_entry_status(table_name, first_year_date.strftime("%Y-%m-%d"), "Successful")

    print("Completed processing DimDate...")
        
populate_dimdate()

DimDate is already loaded for 2013-01-01.
DimDate is already loaded for 2014-01-01.


Completed processing DimDate...


### Populate DimCity table

In [287]:
table_name = "DimCity"

def populate_dimcity():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("Countries")
        load_changed_table("StateProvinces")


        # Step 3: Save available tables
        load_available_table("Countries")
        load_available_table("StateProvinces")


        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    C.CityID AS WWICityID,
                    C.CityName  AS City,
                    SPA.StateProvinceName  AS StateProvince,
                    CA.CountryName AS Country,
                    CA.Continent AS Continent,
                    SPA.SalesTerritory,
                    CA.Region,
                    CA.Subregion,
                    C.Location,
                    C.LatestRecordedPopulation
                FROM 
                    {{sql_table_name}} C LEFT JOIN
                    [__StateProvinces_Available_{{new_cutoff}}] SPA ON
                        SPA.StateProvinceID = C.StateProvinceID LEFT JOIN
                    [__Countries_Available_{{new_cutoff}}] CA ON
                        CA.CountryID = SPA.CountryID
                WHERE
                    {{sql_archive_where}} (
                        DATETIME(C.ValidFrom) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(C.ValidFrom) >= DATETIME('{{new_cutoff}}')
                        ) OR
                        SPA.StateProvinceID IN 
                        (
                            SELECT
                                SPC.StateProvinceID
                            FROM
                                [__StateProvinces_Changed_{{new_cutoff}}] SPC
                        ) OR
                        CA.CountryID IN 
                        (
                            SELECT
                                CC2.CountryID
                            FROM
                                [__Countries_Changed_{{new_cutoff}}] CC2
                        )
                    ) AND
                    (
                        DATETIME('{{new_cutoff}}') <= DATETIME(C.ValidTo) OR
                        DATETIME(C.ValidTo) IS NULL
                    ) AND
                    DATETIME(C.ValidFrom) <= DATETIME('{{new_cutoff}}')
            """.replace(
                "{{new_cutoff}}", 
                new_cutoff
            ).replace(
                "{{last_cutoff}}",
                last_cutoff
            ),
            sql_archive_where="C.CityID NOT IN (SELECT DCT.WWICityID FROM __DimCityTemp DCT) AND ", 
            table_name="Cities", 
            archive_table_name="Cities_Archive", 
            save_table_name="__DimCityTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCity Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimCity
                    (
                        CityKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWICityID INTEGER,
                        City TEXT,
                        StateProvince TEXT,
                        Country TEXT,
                        Continent TEXT,
                        SalesTerritory TEXT,
                        Region TEXT,
                        Subregion TEXT,
                        Location TEXT,
                        LatestRecordedPopulation REAL
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimCity
                    SET 
                        City = __DimCityTemp.City,
                        StateProvince = __DimCityTemp.StateProvince,
                        Country = __DimCityTemp.Country,
                        Continent = __DimCityTemp.Continent,
                        SalesTerritory = __DimCityTemp.SalesTerritory,
                        Region = __DimCityTemp.Region,
                        Subregion = __DimCityTemp.Subregion,
                        Location = __DimCityTemp.Location,
                        LatestRecordedPopulation = __DimCityTemp.LatestRecordedPopulation
                    FROM
                        __DimCityTemp 
                    WHERE
                        DimCity.WWICityID = __DimCityTemp.WWICityID
                """,
                # Add missing data
                """
                    INSERT INTO DimCity
                    (
                        WWICityID,
                        City,
                        StateProvince,
                        Country,
                        Continent,
                        SalesTerritory,
                        Region,
                        Subregion,
                        Location,
                        LatestRecordedPopulation
                    )
                    SELECT 
                        DCT.WWICityID,
                        DCT.City,
                        DCT.StateProvince,
                        DCT.Country,
                        DCT.Continent,
                        DCT.SalesTerritory,
                        DCT.Region,
                        DCT.Subregion,
                        DCT.Location,
                        DCT.LatestRecordedPopulation
                    FROM 
                        __DimCityTemp DCT LEFT JOIN 
                        DimCity DC ON
                            DC.WWICityID  = DCT.WWICityID 
                    WHERE
                        DC.WWICityID  IS NULL
                """
            ],
            db_wh
        )


        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimCity processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimCity processing failed...", e)
    finally:
        print("Completed processing DimCity...")

populate_dimcity()

Successfully created __Countries_Changed table.
Successfully created __StateProvinces_Changed table.

Successfully created __Countries_Available table.
Successfully created __StateProvinces_Available table.


Successfully created __DimCityTemp table.
DimCity processing successful...
Completed processing DimCity...


### Populate DimCustomer Table

In [288]:
table_name = "DimCustomer"

def populate_dimcustomer():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("CustomerCategories")
        load_changed_table("BuyingGroups")
        load_changed_table("PrimaryContacts")


        # Step 3: Save available tables
        load_available_table("CustomerCategories")
        load_available_table("BuyingGroups")
        load_available_table("PrimaryContacts")
        load_available_table("Customers")
        

        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    C.CustomerID AS WWICustomerID,
                    C.CustomerName AS Customer,
                    C2.CustomerName AS BillToCustomer,
                    CC.CustomerCategoryName AS Category,
                    BG.BuyingGroupName AS BuyingGroup,
                    PC.FullName AS PrimaryContact,
                    C.DeliveryPostalCode AS PostalCode
                FROM 
                    {{sql_table_name}} C LEFT JOIN
                    [__Customers_Available_{{new_cutoff}}] C2 ON
                        C2.CustomerID = C.BillToCustomerID LEFT JOIN
                    [__CustomerCategories_Available_{{new_cutoff}}] CC ON
                        CC.CustomerCategoryID = C.CustomerCategoryID LEFT JOIN
                    [__BuyingGroups_Available_{{new_cutoff}}] BG ON
                        BG.BuyingGroupID = C.BuyingGroupID LEFT JOIN
                    [__PrimaryContacts_Available_{{new_cutoff}}] PC ON
                        PC.PersonID = C.PrimaryContactPersonID
                WHERE
                    {{sql_archive_where}} (
                        DATETIME(C.ValidFrom) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(C.ValidFrom) >= DATETIME('{{new_cutoff}}')
                        ) OR
                        C.CustomerCategoryID IN 
                        (
                            SELECT
                                CCC.CustomerCategoryID
                            FROM
                                [__CustomerCategories_Changed_{{new_cutoff}}] CCC
                        ) OR
                        C.BuyingGroupID IN 
                        (
                            SELECT
                                BGC.BuyingGroupID
                            FROM
                                [__BuyingGroups_Changed_{{new_cutoff}}] BGC
                        ) OR
                        C.PrimaryContactPersonID IN 
                        (
                            SELECT
                                PCC.PersonID
                            FROM
                                [__PrimaryContacts_Changed_{{new_cutoff}}] PCC
                        )
                    ) AND
                    (
                        DATETIME('{{new_cutoff}}') <= DATETIME(C.ValidTo) OR
                        DATETIME(C.ValidTo) IS NULL
                    ) AND
                    DATETIME(C.ValidFrom) <= DATETIME('{{new_cutoff}}') 

            """.replace(
                "{{new_cutoff}}", 
                new_cutoff
            ).replace(
                "{{last_cutoff}}",
                last_cutoff
            ),
            sql_archive_where="C.CustomerID NOT IN (SELECT DCT.WWICustomerID FROM __DimCustomerTemp DCT) AND ", 
            table_name="Customers", 
            archive_table_name="Customers_Archive", 
            save_table_name="__DimCustomerTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimCustomer
                    (
                        CustomerKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWICustomerID INTEGER,
                        Customer TEXT,
                        BillToCustomer TEXT,
                        Category TEXT,
                        BuyingGroup TEXT,
                        PrimaryContact TEXT,
                        PostalCode TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimCustomer
                    SET 
                        Customer = __DimCustomerTemp.Customer,
                        BillToCustomer = __DimCustomerTemp.BillToCustomer,
                        Category = __DimCustomerTemp.Category,
                        BuyingGroup = __DimCustomerTemp.BuyingGroup,
                        PrimaryContact = __DimCustomerTemp.PrimaryContact,
                        PostalCode = __DimCustomerTemp.PostalCode
                    FROM
                        __DimCustomerTemp 
                    WHERE
                        DimCustomer.WWICustomerID = __DimCustomerTemp.WWICustomerID
                """,
                # Add missing data
                """
                    INSERT INTO DimCustomer
                    (
                        WWICustomerID,
                        Customer,
                        BillToCustomer,
                        Category,
                        BuyingGroup,
                        PrimaryContact,
                        PostalCode
                    )
                    SELECT 
                        DCT.WWICustomerID,
                        DCT.Customer,
                        DCT.BillToCustomer,
                        DCT.Category,
                        DCT.BuyingGroup,
                        DCT.PrimaryContact,
                        DCT.PostalCode
                    FROM 
                        __DimCustomerTemp DCT LEFT JOIN 
                        DimCustomer DC ON
                            DC.WWICustomerID  = DCT.WWICustomerID 
                    WHERE
                        DC.WWICustomerID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimCustomer processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimCustomer processing failed...", e)
    finally:
        print("Completed processing DimCustomer...")

populate_dimcustomer()

Successfully created __CustomerCategories_Changed table.
Successfully created __BuyingGroups_Changed table.


Successfully created __PrimaryContacts_Changed table.
Successfully created __CustomerCategories_Available table.


Successfully created __BuyingGroups_Available table.
Successfully created __PrimaryContacts_Available table.


Successfully created __Customers_Available table.


Successfully created __DimCustomerTemp table.
DimCustomer processing successful...
Completed processing DimCustomer...


### Populate DimEmployee table

In [289]:
table_name = "DimEmployee"

def populate_dimemployee():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("Employees")
        

        # Step 3: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    P.PersonID AS WWIEmployeeID,
                    P.FullName AS Employee,
                    P.PreferredName AS PreferredName,
                    P.IsSalesperson AS IsSalesPerson,
                    P.Photo
                FROM 
                    {{sql_table_name}} P 
                WHERE
                    {{sql_archive_where}}
                    P.PersonID IN 
                    (
                        SELECT
                            EC.PersonID
                        FROM
                            [__Employees_Changed_{{new_cutoff}}] EC
                    ) 
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ),
            sql_archive_where="P.PersonID NOT IN (SELECT DET.WWIEmployeeID FROM __DimEmployeeTemp DET) AND ", 
            table_name="People", 
            archive_table_name="People_Archive", 
            save_table_name="__DimEmployeeTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimEmployee
                    (
                        EmployeeKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWIEmployeeID INTEGER,
                        Employee TEXT,
                        PreferredName TEXT,
                        IsSalesPerson INTEGER,
                        Photo TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimEmployee
                    SET 
                        Employee = __DimEmployeeTemp.Employee,
                        PreferredName = __DimEmployeeTemp.PreferredName,
                        IsSalesPerson = __DimEmployeeTemp.IsSalesPerson,
                        Photo = __DimEmployeeTemp.Photo
                    FROM
                        __DimEmployeeTemp 
                    WHERE
                        DimEmployee.WWIEmployeeID = __DimEmployeeTemp.WWIEmployeeID
                """,
                # Add missing data
                """
                    INSERT INTO DimEmployee
                    (
                        WWIEmployeeID,
                        Employee,
                        PreferredName,
                        IsSalesPerson,
                        Photo
                    )
                    SELECT 
                        DET.WWIEmployeeID,
                        DET.Employee,
                        DET.PreferredName,
                        DET.IsSalesPerson,
                        DET.Photo
                    FROM 
                        __DimEmployeeTemp DET LEFT JOIN 
                        DimEmployee DE ON
                            DE.WWIEmployeeID  = DET.WWIEmployeeID 
                    WHERE
                        DE.WWIEmployeeID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimEmployee processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimEmployee processing failed...", e)
    finally:
        print("Completed processing DimEmployee...")

populate_dimemployee()

Successfully created __Employees_Changed table.


Successfully created __DimEmployeeTemp table.
DimEmployee processing successful...
Completed processing DimEmployee...


### Populate DimPaymentMethod table

In [290]:
table_name = "DimPaymentMethod"

def populate_dimpaymentmethod():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("PaymentMethods")
        

        # Step 3: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    PM.PaymentMethodID AS WWIPaymentMethodID,
                    PM.PaymentMethodName AS PaymentMethod
                FROM 
                    {{sql_table_name}} PM 
                WHERE
                    {{sql_archive_where}}
                    PM.PaymentMethodID IN 
                    (
                        SELECT
                            PMC.PaymentMethodID
                        FROM
                            [__PaymentMethods_Changed_{{new_cutoff}}] PMC
                    ) 
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ),
            sql_archive_where="PM.PaymentMethodID NOT IN (SELECT DPMT.WWIPaymentMethodID FROM __DimPaymentMethodTemp DPMT) AND ", 
            table_name="PaymentMethods", 
            archive_table_name="PaymentMethods_Archive", 
            save_table_name="__DimPaymentMethodTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimPaymentMethod
                    (
                        PaymentMethodKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWIPaymentMethodID INTEGER,
                        PaymentMethod TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimPaymentMethod
                    SET 
                        PaymentMethod = __DimPaymentMethodTemp.PaymentMethod
                    FROM
                        __DimPaymentMethodTemp 
                    WHERE
                        DimPaymentMethod.WWIPaymentMethodID = __DimPaymentMethodTemp.WWIPaymentMethodID
                """,
                # Add missing data
                """
                    INSERT INTO DimPaymentMethod
                    (
                        WWIPaymentMethodID,
                        PaymentMethod
                    )
                    SELECT 
                        DPMT.WWIPaymentMethodID,
                        DPMT.PaymentMethod
                    FROM 
                        __DimPaymentMethodTemp DPMT LEFT JOIN 
                        DimPaymentMethod DPM ON
                            DPM.WWIPaymentMethodID  = DPMT.WWIPaymentMethodID 
                    WHERE
                        DPM.WWIPaymentMethodID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimPaymentMethod processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimPaymentMethod processing failed...", e)
    finally:
        print("Completed processing DimPaymentMethod...")

populate_dimpaymentmethod()

Successfully created __PaymentMethods_Changed table.


Successfully created __DimPaymentMethodTemp table.
DimPaymentMethod processing successful...
Completed processing DimPaymentMethod...


### Populate DimStockItem table

In [291]:
table_name = "DimStockItem"

def populate_dimstockitem():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("PackageTypes")
        load_changed_table("Colors")


        # Step 3: Save available tables
        # Package Types
        load_available_table("PackageTypes")
        load_available_table("Colors")
        

        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    SI.StockItemID AS WWIStockItemID, 
                    SI.StockItemName AS StockItem,
                    CA.ColorName AS Color,
                    SPTA.PackageTypeName AS SellingPackage,
                    BPTA.PackageTypeName AS BuyingPackage, 
                    SI.Brand, 
                    SI.Size, 
                    SI.LeadTimeDays, 
                    SI.QuantityPerOuter,
                    SI.IsChillerStock, 
                    SI.Barcode, 
                    SI.TaxRate, 
                    SI.UnitPrice, 
                    SI.RecommendedRetailPrice,
                    SI.TypicalWeightPerUnit, 
                    SI.Photo
                FROM 
                    {{sql_table_name}} SI LEFT JOIN
                    [__PackageTypes_Available_{{new_cutoff}}] SPTA ON
                        SI.UnitPackageID = SPTA.PackageTypeID LEFT JOIN 
                    [__PackageTypes_Available_{{new_cutoff}}] BPTA ON
                        SI.OuterPackageID = BPTA.PackageTypeID LEFT JOIN
                    [__Colors_Available_{{new_cutoff}}] CA ON
                        SI.ColorID = CA.ColorID
                WHERE
                    {{sql_archive_where}} (
                        DATETIME(SI.ValidFrom) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(SI.ValidFrom) >= DATETIME('{{new_cutoff}}')
                        ) OR
                        SI.UnitPackageID IN 
                        (
                            SELECT
                                PTC.PackageTypeID
                            FROM
                                [__PackageTypes_Changed_{{new_cutoff}}] PTC
                        ) OR
                        SI.OuterPackageID IN 
                        (
                            SELECT
                                PTC.PackageTypeID
                            FROM
                                [__PackageTypes_Changed_{{new_cutoff}}] PTC
                        ) OR
                        SI.ColorID IN 
                        (
                            SELECT
                                BGC.ColorID
                            FROM
                                [__Colors_Changed_{{new_cutoff}}] BGC
                        ) 
                    ) AND
                    (
                        DATETIME('{{new_cutoff}}') <= DATETIME(SI.ValidTo) OR
                        DATETIME(SI.ValidTo) IS NULL
                    ) AND
                    DATETIME(SI.ValidFrom) <= DATETIME('{{new_cutoff}}') 

            """.replace(
                "{{new_cutoff}}", 
                new_cutoff
            ).replace(
                "{{last_cutoff}}",
                last_cutoff
            ),
            sql_archive_where="SI.StockItemID NOT IN (SELECT DSIT.WWIStockItemID FROM __DimStockItemTemp DSIT) AND ", 
            table_name="StockItems", 
            archive_table_name="StockItems_Archive", 
            save_table_name="__DimStockItemTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimStockItem Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimStockItem
                    (
                        StockItemKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWIStockItemID INTEGER, 
                        StockItem TEXT,
                        Color TEXT,
                        SellingPackage TEXT,
                        BuyingPackage TEXT, 
                        Brand TEXT, 
                        Size TEXT, 
                        LeadTimeDays INTEGER, 
                        QuantityPerOuter INTEGER,
                        IsChillerStock INTEGER, 
                        Barcode TEXT, 
                        TaxRate REAL, 
                        UnitPrice REAL, 
                        RecommendedRetailPrice REAL,
                        TypicalWeightPerUnit REAL, 
                        Photo TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimStockItem
                    SET 
                        StockItem = __DimStockItemTemp.StockItem,
                        Color = __DimStockItemTemp.Color,
                        SellingPackage = __DimStockItemTemp.SellingPackage,
                        BuyingPackage = __DimStockItemTemp.BuyingPackage, 
                        Brand = __DimStockItemTemp.Brand, 
                        Size = __DimStockItemTemp.Size, 
                        LeadTimeDays = __DimStockItemTemp.LeadTimeDays, 
                        QuantityPerOuter = __DimStockItemTemp.QuantityPerOuter,
                        IsChillerStock = __DimStockItemTemp.IsChillerStock, 
                        Barcode = __DimStockItemTemp.Barcode, 
                        TaxRate = __DimStockItemTemp.TaxRate, 
                        UnitPrice = __DimStockItemTemp.UnitPrice, 
                        RecommendedRetailPrice = __DimStockItemTemp.RecommendedRetailPrice,
                        TypicalWeightPerUnit = __DimStockItemTemp.TypicalWeightPerUnit, 
                        Photo = __DimStockItemTemp.Photo
                    FROM
                        __DimStockItemTemp
                    WHERE
                        DimStockItem.WWIStockItemID = __DimStockItemTemp.WWIStockItemID
                """,
                # Add missing data
                """
                    INSERT INTO DimStockItem
                    (
                        WWIStockItemID, 
                        StockItem,
                        Color,
                        SellingPackage,
                        BuyingPackage, 
                        Brand, 
                        Size, 
                        LeadTimeDays, 
                        QuantityPerOuter,
                        IsChillerStock, 
                        Barcode, 
                        TaxRate, 
                        UnitPrice, 
                        RecommendedRetailPrice,
                        TypicalWeightPerUnit, 
                        Photo
                    )
                    SELECT 
                        DSIT.WWIStockItemID, 
                        DSIT.StockItem,
                        DSIT.Color,
                        DSIT.SellingPackage,
                        DSIT.BuyingPackage, 
                        DSIT.Brand, 
                        DSIT.Size, 
                        DSIT.LeadTimeDays, 
                        DSIT.QuantityPerOuter,
                        DSIT.IsChillerStock, 
                        DSIT.Barcode, 
                        DSIT.TaxRate, 
                        DSIT.UnitPrice, 
                        DSIT.RecommendedRetailPrice,
                        DSIT.TypicalWeightPerUnit, 
                        DSIT.Photo
                    FROM 
                        __DimStockItemTemp DSIT LEFT JOIN 
                        DimStockItem DSI ON
                            DSI.WWIStockItemID = DSIT.WWIStockItemID 
                    WHERE
                        DSI.WWIStockItemID IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimStockItem processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimStockItem processing failed...", e)
    finally:
        print("Completed processing DimStockItem...")

populate_dimstockitem()

Successfully created __PackageTypes_Changed table.
Successfully created __Colors_Changed table.


Successfully created __PackageTypes_Available table.


Successfully created __Colors_Available table.


Successfully created __DimStockItemTemp table.
DimStockItem processing successful...
Completed processing DimStockItem...


### Populate DimSupplier table

In [292]:
table_name = "DimSupplier"

def populate_dimsupplier():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("Suppliers")
        load_changed_table("PrimaryContacts")
        

        # Step 3: Save available tables
        load_available_table("SupplierCategories")
        load_available_table("PrimaryContacts")
        

        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    S.SupplierID AS WWISupplierID, 
                    S.SupplierName AS Supplier, 
                    SCA.SupplierCategoryName AS Category, 
                    PCA.FullName AS PrimaryContact, 
                    S.SupplierReference,
                    S.PaymentDays, 
                    S.DeliveryPostalCode AS PostalCode
                FROM 
                    {{sql_table_name}} S LEFT JOIN
                    [__SupplierCategories_Available_{{new_cutoff}}] SCA ON
                        S.SupplierCategoryID = SCA.SupplierCategoryID LEFT JOIN 
                    [__PrimaryContacts_Available_{{new_cutoff}}] PCA ON
                        S.PrimaryContactPersonID = PCA.PersonID 
                WHERE
                    {{sql_archive_where}} (
                        DATETIME(S.ValidFrom) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(S.ValidFrom) >= DATETIME('{{new_cutoff}}')
                        ) OR
                        S.SupplierCategoryID IN 
                        (
                            SELECT
                                SCC.SupplierCategoryID
                            FROM
                                [__SupplierCategories_Changed_{{new_cutoff}}] SCC
                        ) OR
                        S.PrimaryContactPersonID IN 
                        (
                            SELECT
                                PCC.PersonID
                            FROM
                                [__PrimaryContacts_Changed_{{new_cutoff}}] PCC
                        ) 
                    ) AND
                    (
                        DATETIME('{{new_cutoff}}') <= DATETIME(S.ValidTo) OR
                        DATETIME(S.ValidTo) IS NULL
                    ) AND
                    DATETIME(S.ValidFrom) <= DATETIME('{{new_cutoff}}') 

            """.replace(
                "{{new_cutoff}}", 
                new_cutoff
            ).replace(
                "{{last_cutoff}}",
                last_cutoff
            ),
            sql_archive_where="S.SupplierID NOT IN (SELECT DST.WWISupplierID FROM __DimSupplierTemp DST) AND ", 
            table_name="Suppliers", 
            archive_table_name="Suppliers_Archive", 
            save_table_name="__DimSupplierTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimStockItem Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimSupplier

                    (
                        SupplierKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWISupplierID INTEGER, 
                        Supplier TEXT, 
                        Category TEXT, 
                        PrimaryContact TEXT, 
                        SupplierReference TEXT,
                        PaymentDays INTEGER, 
                        PostalCode TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimSupplier
                    SET 
                        Supplier = __DimSupplierTemp.Supplier, 
                        Category = __DimSupplierTemp.Category, 
                        PrimaryContact = __DimSupplierTemp.PrimaryContact, 
                        SupplierReference = __DimSupplierTemp.SupplierReference,
                        PaymentDays = __DimSupplierTemp.PaymentDays, 
                        PostalCode = __DimSupplierTemp.PostalCode
                    FROM
                        __DimSupplierTemp
                    WHERE
                        DimSupplier.WWISupplierID = __DimSupplierTemp.WWISupplierID
                """,
                # Add missing data
                """
                    INSERT INTO DimSupplier
                    (
                        WWISupplierID, 
                        Supplier, 
                        Category, 
                        PrimaryContact, 
                        SupplierReference,
                        PaymentDays, 
                        PostalCode
                    )
                    SELECT 
                        DST.WWISupplierID, 
                        DST.Supplier, 
                        DST.Category, 
                        DST.PrimaryContact, 
                        DST.SupplierReference,
                        DST.PaymentDays, 
                        DST.PostalCode
                    FROM 
                        __DimSupplierTemp DST LEFT JOIN 
                        DimSupplier DS ON
                            DS.WWISupplierID = DST.WWISupplierID 
                    WHERE
                        DS.WWISupplierID IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimSupplier processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimSupplier processing failed...", e)
    finally:
        print("Completed processing DimSupplier...")

populate_dimsupplier()

Successfully created __SupplierCategories_Changed table.
Table exists __PrimaryContacts_Changed_2025-01-01


Successfully created __SupplierCategories_Available table.
Table exists __PrimaryContacts_Available_2025-01-01


Successfully created __DimSupplierTemp table.
DimSupplier processing successful...
Completed processing DimSupplier...


### Populate DimTransactionType table

In [293]:
table_name = "DimTransactionType"

def populate_dimtransactiontype():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("TransactionTypes")
        

        # Step 3: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    TT.TransactionTypeID AS WWITransactionTypeID,
                    TT.TransactionTypeName AS TransactionType
                FROM 
                    {{sql_table_name}} TT 
                WHERE
                    {{sql_archive_where}}
                    Tt.TransactionTypeID IN 
                    (
                        SELECT
                            TTC.TransactionTypeID
                        FROM
                            [__TransactionTypes_Changed_{{new_cutoff}}] TTC
                    ) 
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ),
            sql_archive_where="TT.TransactionTypeID NOT IN (SELECT DTTT.WWITransactionTypeID FROM __DimTransactionTypeTemp DTTT) AND ", 
            table_name="TransactionTypes", 
            archive_table_name="TransactionTypes_Archive", 
            save_table_name="__DimTransactionTypeTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS DimTransactionType
                    (
                        TransactionTypeKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        WWITransactionTypeID INTEGER,
                        TransactionType TEXT
                    )
                """,
                # Update existing data
                """
                    UPDATE 
                        DimTransactionType
                    SET 
                        TransactionType = __DimTransactionTypeTemp.TransactionType
                    FROM
                        __DimTransactionTypeTemp 
                    WHERE
                        DimTransactionType.WWITransactionTypeID = __DimTransactionTypeTemp.WWITransactionTypeID
                """,
                # Add missing data
                """
                    INSERT INTO DimTransactionType
                    (
                        WWITransactionTypeID,
                        TransactionType
                    )
                    SELECT 
                        DTTT.WWITransactionTypeID,
                        DTTT.TransactionType
                    FROM 
                        __DimTransactionTypeTemp DTTT LEFT JOIN 
                        DimTransactionType DTT ON
                            DTT.WWITransactionTypeID  = DTTT.WWITransactionTypeID 
                    WHERE
                        DTT.WWITransactionTypeID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("DimTransactionType processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("DimTransactionType processing failed...", e)
    finally:
        print("Completed processing DimTransactionType...")

populate_dimtransactiontype()

Successfully created __TransactionTypes_Changed table.


Successfully created __DimTransactionTypeTemp table.
DimTransactionType processing successful...
Completed processing DimTransactionType...


### Populate FctMovement table

In [294]:
table_name = "FctMovement"

def populate_fctmovement():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # Step 2: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    TransactionOccurredWhen AS DateKey,
                    CAST(NULL AS INTEGER) AS StockItemKey,
                    CAST(NULL AS INTEGER) AS CustomerKey,
                    CAST(NULL AS INTEGER) AS SupplierKey,
                    CAST(NULL AS INTEGER) TransactionTypeKey,
                    SIT.StockItemID AS WWIStockItemID,
                    SIT.CustomerID AS WWICustomerID,
                    SIT.SupplierID AS WWISupplierID,
                    SIT.TransactionTypeID AS WWITransactionTypeID,
                    SIT.StockItemTransactionID AS WWIStockItemTransactionID,
                    SIT.InvoiceID AS WWIInvoiceID,
                    SIT.PurchaseOrderID AS WWIPurchaseOrderID,
                    SIT.Quantity
                FROM 
                    StockItemTransactions SIT 
                WHERE
                    (
                        DATETIME(SIT.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(SIT.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                        )
                    ) AND
                    DATETIME(SIT.LastEditedWhen) <= DATETIME('{{new_cutoff}}')
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ).replace(
               "{{last_cutoff}}", 
                last_cutoff
            ).replace(
               "{{initial_load}}", 
                initial_load
            ),
            sql_archive_where="", 
            table_name="StockItemTransactions", 
            archive_table_name="", 
            save_table_name="__FctMovementTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctMovement
                    (
                        MovementKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        DateKey TEXT,
                        StockItemKey INTEGER,
                        CustomerKey INTEGER,
                        SupplierKey INTEGER,
                        TransactionTypeKey INTEGER,
                        WWIStockItemTransactionID INTEGER,
                        WWIInvoiceID INTEGER,
                        WWIPurchaseOrderID INTEGER,
                        Quantity INTEGER
                    )
                """,
                # Update StockItemKey
                """
                    UPDATE
                        __FctMovementTemp
                    SET
                        StockItemKey = DimStockItem.StockItemKey
                    FROM
                        DimStockItem
                    WHERE
                        DimStockItem.WWIStockItemID = __FctMovementTemp.WWIStockItemID
                """,
                # Update CustomerKey
                """
                    UPDATE
                        __FctMovementTemp
                    SET
                        CustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctMovementTemp.WWICustomerID
                """,
                # Update SupplierKey
                """
                    UPDATE
                        __FctMovementTemp
                    SET
                        SupplierKey = DimSupplier.SupplierKey
                    FROM
                        DimSupplier
                    WHERE
                        DimSupplier.WWISupplierID = __FctMovementTemp.WWISupplierID
                """,
                # Update TransactionTypeKey
                """
                    UPDATE
                        __FctMovementTemp
                    SET
                        TransactionTypeKey = DimTransactionType.TransactionTypeKey
                    FROM
                        DimTransactionType
                    WHERE
                        DimTransactionType.WWITransactionTypeID = __FctMovementTemp.WWITransactionTypeID
                """,
                # Update existing data
                """
                    UPDATE 
                        FctMovement
                    SET 
                        DateKey = SUBSTR(__FctMovementTemp.DateKey, 1, 10),
                        StockItemKey = __FctMovementTemp.StockItemKey,
                        CustomerKey = __FctMovementTemp.CustomerKey,
                        SupplierKey = __FctMovementTemp.SupplierKey,
                        TransactionTypeKey = __FctMovementTemp.TransactionTypeKey,
                        WWIInvoiceID = __FctMovementTemp.WWIInvoiceID,
                        WWIPurchaseOrderID = __FctMovementTemp.WWIPurchaseOrderID,
                        Quantity = __FctMovementTemp.Quantity
                    FROM
                        __FctMovementTemp
                    WHERE
                        FctMovement.WWIStockItemTransactionID = __FctMovementTemp.WWIStockItemTransactionID
                """,
                # Add missing data
                """
                    INSERT INTO FctMovement
                    (
                        DateKey,
                        StockItemKey,
                        CustomerKey,
                        SupplierKey,
                        TransactionTypeKey,
                        WWIStockItemTransactionID,
                        WWIInvoiceID,
                        WWIPurchaseOrderID,
                        Quantity
                    )
                    SELECT 
                        SUBSTR(FMT.DateKey, 1, 10),
                        FMT.StockItemKey,
                        FMT.CustomerKey,
                        FMT.SupplierKey,
                        FMT.TransactionTypeKey,
                        FMT.WWIStockItemTransactionID,
                        FMT.WWIInvoiceID,
                        FMT.WWIPurchaseOrderID,
                        FMT.Quantity
                    FROM 
                        __FctMovementTemp FMT LEFT JOIN 
                        FctMovement FM ON
                            FM.WWIStockItemTransactionID  = FMT.WWIStockItemTransactionID 
                    WHERE
                        FM.WWIStockItemTransactionID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctMovement processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("FctMovement processing failed...", e)
    finally:
        print("Completed processing FctMovement...")

populate_fctmovement()

Successfully created __FctMovementTemp table.


FctMovement processing successful...
Completed processing FctMovement...


Populate FctOrder table

In [295]:
table_name = "FctOrder"

def populate_fctorder():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("PackageTypes")


        # Step 3: Save available tables
        load_available_table("PackageTypes")
        load_available_table("Customers")


        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    NULL AS CityKey,
                    NULL AS CustomerKey,
                    NULL AS StockItemKey,
                    O.OrderDate AS OrderDateKey,
                    OL.PickingCompletedWhen AS PickedDateKey,
                    NULL AS SalesPersonKey,
                    NULL AS PickerKey,
                    O.OrderID AS WWIOrderID,
                    OL.OrderLineID AS WWIOrderLineID,
                    O.BackorderOrderID AS WWIBackorderID,
                    C.DeliveryCityID AS WWICityID,
                    C.CustomerID AS WWICustomerID,
                    OL.StockItemID AS WWIStockItemID,
                    O.SalespersonPersonID AS WWISalesPersonID,
                    O.PickedByPersonID AS WWIPickerID,
                    OL.Description,
                    PT.PackageTypeName AS Package,
                    OL.Quantity,
                    OL.UnitPrice,
                    OL.TaxRate,
                    ROUND(OL.Quantity * OL.UnitPrice, 2) AS TotalExcludingTax,
                    ROUND((OL.Quantity * OL.UnitPrice * OL.TaxRate) / 100.0, 2) TaxAmount,
                    (
                        ROUND(OL.Quantity * OL.UnitPrice, 2) + 
                        ROUND((OL.Quantity * OL.UnitPrice * OL.TaxRate) / 100.0, 2)
                    ) AS TotalIncludingTax
                FROM 
                    Orders O JOIN 
                    OrderLines OL ON 
                        O.OrderID = OL.OrderID JOIN 
                    [__PackageTypes_Available_{{new_cutoff}}] PT ON
                        PT.PackageTypeID = OL.PackageTypeID JOIN
                    [__Customers_Available_{{new_cutoff}}] C ON 
                        C.CustomerID = O.CustomerID
                WHERE 
                    (
                        DATETIME(O.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        DATETIME(OL.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            (
                                DATETIME(O.LastEditedWhen) >= DATETIME('{{new_cutoff}}') OR
                                DATETIME(OL.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                            )
                        ) OR
                        OL.PackageTypeID IN 
                        (
                            SELECT 
                                PTC.PackageTypeID 
                            FROM 
                                [__PackageTypes_Changed_{{new_cutoff}}] PTC
                        )
                    ) AND
                    (
                        DATETIME(O.LastEditedWhen) <= DATETIME('{{new_cutoff}}') OR
                        DATETIME(OL.LastEditedWhen) <= DATETIME('{{new_cutoff}}')
                    )
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ).replace(
               "{{last_cutoff}}", 
                last_cutoff
            ).replace(
               "{{initial_load}}", 
                initial_load
            ),
            sql_archive_where="", 
            table_name="Orders", 
            archive_table_name="", 
            save_table_name="__FctOrderTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctOrder
                    (
                        OrderKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        CityKey INTEGER,
                        CustomerKey INTEGER,
                        StockItemKey INTEGER,
                        OrderDateKey TEXT,
                        PickedDateKey TEXT,
                        SalesPersonKey INTEGER,
                        PickerKey INTEGER,
                        WWIOrderID INTEGER,
                        WWIOrderLineID INTEGER,
                        WWIBackorderID INTEGER,
                        Description TEXT,
                        Package TEXT,
                        Quantity INTEGER,
                        UnitPrice REAL,
                        TaxRate REAL,
                        TotalExcludingTax REAL,
                        TaxAmount REAL,
                        TotalIncludingTax REAL
                    )
                """,
                # Update CityKey
                """
                    UPDATE
                        __FctOrderTemp
                    SET
                        CityKey = DimCity.CityKey
                    FROM
                        DimCity
                    WHERE
                        DimCity.WWICityID = __FctOrderTemp.WWICityID
                """,
                # Update CustomerKey
                """
                    UPDATE
                        __FctOrderTemp
                    SET
                        CustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctOrderTemp.WWICustomerID
                """,
                # Update StockItemKey
                """
                    UPDATE
                        __FctOrderTemp
                    SET
                        StockItemKey = DimStockItem.StockItemKey
                    FROM
                        DimStockItem
                    WHERE
                        DimStockItem.WWIStockItemID = __FctOrderTemp.WWIStockItemID
                """,
                # Update SalesPersonKey
                """
                    UPDATE
                        __FctOrderTemp
                    SET
                        SalesPersonKey = DimEmployee.EmployeeKey
                    FROM
                        DimEmployee
                    WHERE
                        DimEmployee.WWIEmployeeID = __FctOrderTemp.WWISalesPersonID
                """,
                # Update PickerKey
                """
                    UPDATE
                        __FctOrderTemp
                    SET
                        PickerKey = DimEmployee.EmployeeKey
                    FROM
                        DimEmployee
                    WHERE
                        DimEmployee.WWIEmployeeID = __FctOrderTemp.WWIPickerID
                """,
                # Update existing data
                """
                    UPDATE 
                        FctOrder
                    SET 
                        CityKey = __FctOrderTemp.CityKey,
                        CustomerKey = __FctOrderTemp.CustomerKey,
                        StockItemKey = __FctOrderTemp.StockItemKey,
                        OrderDateKey = SUBSTR(__FctOrderTemp.OrderDateKey, 1, 10),
                        PickedDateKey = SUBSTR(__FctOrderTemp.PickedDateKey, 1, 10),
                        SalesPersonKey = __FctOrderTemp.SalesPersonKey,
                        PickerKey = __FctOrderTemp.PickerKey,
                        WWIOrderID = __FctOrderTemp.WWIOrderID,
                        WWIOrderLineID = __FctOrderTemp.WWIOrderLineID,
                        WWIBackorderID = __FctOrderTemp.WWIBackorderID,
                        Description = __FctOrderTemp.Description,
                        Package = __FctOrderTemp.Package,
                        Quantity = __FctOrderTemp.Quantity,
                        UnitPrice = __FctOrderTemp.UnitPrice,
                        TaxRate = __FctOrderTemp.TaxRate,
                        TotalExcludingTax = __FctOrderTemp.TotalExcludingTax,
                        TaxAmount = __FctOrderTemp.TaxAmount,
                        TotalIncludingTax = __FctOrderTemp.TotalIncludingTax
                    FROM
                        __FctOrderTemp
                    WHERE
                        FctOrder.WWIOrderID = __FctOrderTemp.WWIOrderID AND
                        FctOrder.WWIOrderLineID = __FctOrderTemp.WWIOrderLineID
                """,
                # Add missing data
                """
                    INSERT INTO FctOrder
                    (
                        CityKey,
                        CustomerKey,
                        StockItemKey,
                        OrderDateKey,
                        PickedDateKey,
                        SalesPersonKey,
                        PickerKey,
                        WWIOrderID,
                        WWIOrderLineID,
                        WWIBackorderID,
                        Description,
                        Package,
                        Quantity,
                        UnitPrice,
                        TaxRate,
                        TotalExcludingTax,
                        TaxAmount,
                        TotalIncludingTax
                    )
                    SELECT 
                        FOT.CityKey,
                        FOT.CustomerKey,
                        FOT.StockItemKey,
                        FOT.OrderDateKey,
                        FOT.PickedDateKey,
                        FOT.SalesPersonKey,
                        FOT.PickerKey,
                        FOT.WWIOrderID,
                        FOT.WWIOrderLineID,
                        FOT.WWIBackorderID,
                        FOT.Description,
                        FOT.Package,
                        FOT.Quantity,
                        FOT.UnitPrice,
                        FOT.TaxRate,
                        FOT.TotalExcludingTax,
                        FOT.TaxAmount,
                        FOT.TotalIncludingTax
                    FROM 
                        __FctOrderTemp FOT LEFT JOIN 
                        FctOrder FO ON
                            FO.WWIOrderID  = FOT.WWIOrderID AND
                            FO.WWIOrderLineID = FOT.WWIOrderLineID 
                    WHERE
                        FO.WWIOrderID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctOrder processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("FctOrder processing failed...", e)
    finally:
        print("Completed processing FctOrder...")

populate_fctorder()

Table exists __PackageTypes_Changed_2025-01-01
Table exists __PackageTypes_Available_2025-01-01
Table exists __Customers_Available_2025-01-01


Successfully created __FctOrderTemp table.


FctOrder processing successful...
Completed processing FctOrder...


### Populate FctPurchase table

In [296]:
table_name = "FctPurchase"

def populate_fctpurchase():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("PackageTypes")


        # Step 3: Save available tables
        load_available_table("PackageTypes")
        load_available_table("StockItems")


        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    PO.OrderDate AS DateKey,
                    NULL AS SupplierKey,
                    NULL AS StockItemKey,
                    PO.PurchaseOrderID AS WWIPurchaseOrderID,
                    POL.PurchaseOrderLineID AS WWIPurchaseOrderLineID,
                    PO.SupplierID AS WWISupplierID,
                    POL.StockItemID AS WWIStockItemID,
                    POL.OrderedOuters AS OrderedOuters,
                    POL.OrderedOuters * SI.QuantityPerOuter AS OrderedQuantity,
                    POL.ReceivedOuters AS ReceivedOuters,
                    PT.PackageTypeName AS Package,
                    POL.IsOrderLineFinalized AS IsOrderFinalized
                FROM 
                    PurchaseOrders PO JOIN 
                    PurchaseOrderLines AS POL
                        ON PO.PurchaseOrderID = POL.PurchaseOrderID JOIN 
                    [__StockItems_Available_{{new_cutoff}}] AS SI
                        ON POL.StockItemID = SI.StockItemID JOIN 
                    [__PackageTypes_Available_{{new_cutoff}}] AS PT
                        ON POL.PackageTypeID = PT.PackageTypeID
                WHERE 
                    (
                        DATETIME(PO.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        DATETIME(POL.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            (
                                DATETIME(PO.LastEditedWhen) >= DATETIME('{{new_cutoff}}') OR
                                DATETIME(POL.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                            )
                        ) OR
                        POL.PackageTypeID IN 
                        (
                            SELECT 
                                PTC.PackageTypeID 
                            FROM 
                                [__PackageTypes_Changed_{{new_cutoff}}] PTC
                        )
                    ) AND
                    (
                        DATETIME(PO.LastEditedWhen) <= DATETIME('{{new_cutoff}}') OR
                        DATETIME(POL.LastEditedWhen) <= DATETIME('{{new_cutoff}}')
                    )
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ).replace(
               "{{last_cutoff}}", 
                last_cutoff
            ).replace(
               "{{initial_load}}", 
                initial_load
            ),
            sql_archive_where="", 
            table_name="PurchaseOrders", 
            archive_table_name="", 
            save_table_name="__FctPurchaseTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctPurchase
                    (
                        PurchaseKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        DateKey TEXT,
                        SupplierKey INTEGER,
                        StockItemKey INTEGER,
                        WWIPurchaseOrderID INTEGER,
                        WWIPurchaseOrderLineID INTEGER,
                        OrderedOuters INTEGER,
                        OrderedQuantity INTEGER,
                        ReceivedOuters INTEGER,
                        Package TEXT,
                        IsOrderFinalized INTEGER
                    )
                """,
                # Update SupplierKey
                """
                    UPDATE
                        __FctPurchaseTemp
                    SET
                        SupplierKey = DimSupplier.SupplierKey
                    FROM
                        DimSupplier
                    WHERE
                        DimSupplier.WWISupplierID = __FctPurchaseTemp.WWISupplierID
                """,
                # Update StockItemKey
                """
                    UPDATE
                        __FctPurchaseTemp
                    SET
                        StockItemKey = DimStockItem.StockItemKey
                    FROM
                        DimStockItem
                    WHERE
                        DimStockItem.WWIStockItemID = __FctPurchaseTemp.WWIStockItemID
                """,
                # Update existing data
                """
                    UPDATE 
                        FctPurchase
                    SET 
                        DateKey = SUBSTR(__FctPurchaseTemp.DateKey, 1, 10),
                        SupplierKey = __FctPurchaseTemp.SupplierKey,
                        StockItemKey = __FctPurchaseTemp.StockItemKey,
                        OrderedOuters = __FctPurchaseTemp.OrderedOuters,
                        OrderedQuantity = __FctPurchaseTemp.OrderedQuantity,
                        ReceivedOuters = __FctPurchaseTemp.ReceivedOuters,
                        Package = __FctPurchaseTemp.Package,
                        IsOrderFinalized = __FctPurchaseTemp.IsOrderFinalized
                    FROM
                        __FctPurchaseTemp
                    WHERE
                        FctPurchase.WWIPurchaseOrderID = __FctPurchaseTemp.WWIPurchaseOrderID AND
                        FctPurchase.WWIPurchaseOrderLineID = __FctPurchaseTemp.WWIPurchaseOrderLineID
                """,
                # Add missing data
                """
                    INSERT INTO FctPurchase
                    (
                        DateKey,
                        SupplierKey,
                        StockItemKey,
                        WWIPurchaseOrderID,
                        WWIPurchaseOrderLineID,
                        OrderedOuters,
                        OrderedQuantity,
                        ReceivedOuters,
                        Package,
                        IsOrderFinalized
                    )
                    SELECT 
                        SUBSTR(FPT.DateKey, 1, 10),
                        FPT.SupplierKey,
                        FPT.StockItemKey,
                        FPT.WWIPurchaseOrderID,
                        FPT.WWIPurchaseOrderLineID,
                        FPT.OrderedOuters,
                        FPT.OrderedQuantity,
                        FPT.ReceivedOuters,
                        FPT.Package,
                        FPT.IsOrderFinalized
                    FROM 
                        __FctPurchaseTemp FPT LEFT JOIN 
                        FctPurchase FP ON
                            FP.WWIPurchaseOrderID  = FPT.WWIPurchaseOrderID AND
                            FP.WWIPurchaseOrderLineID  = FPT.WWIPurchaseOrderLineID
                    WHERE
                        FP.WWIPurchaseOrderID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctPurchase processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("FctPurchase processing failed...", e)
    finally:
        print("Completed processing FctPurchase...")

populate_fctpurchase()

Table exists __PackageTypes_Changed_2025-01-01
Table exists __PackageTypes_Available_2025-01-01


Successfully created __StockItems_Available table.


Successfully created __FctPurchaseTemp table.
FctPurchase processing successful...
Completed processing FctPurchase...


### Populate FctSale table

In [297]:
table_name = "FctSale"

def populate_fctsale():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)


        # STEP 2: Save updated tables
        load_changed_table("PackageTypes")


        # Step 3: Save available tables
        load_available_table("PackageTypes")
        load_available_table("StockItems")
        load_available_table("Customers")


        # Step 4: Save temp table
        save_temp_table(
            sql_template="""
                SELECT
                    NULL AS CityKey,
                    NULL AS CustomerKey,
                    NULL AS BillToCustomerKey,
                    NULL AS StockItemKey,
                    I.InvoiceDate AS InvoiceDateKey,
                    I.ConfirmedDeliveryTime AS DeliveryDateKey,
                    NULL AS SalesPersonKey,
                    I.InvoiceID AS WWIInvoiceID,
                    IL.InvoiceLineID AS WWIInvoiceLineID,
                    C.DeliveryCityID AS WWICityID,
                    I.CustomerID AS WWICustomerID,
                    I.BillToCustomerID AS WWIBillToCustomerID,
                    IL.StockItemID AS WWIStockItemID,
                    I.SalespersonPersonID AS WWISalesPersonID,
                    IL.Description,
                    PT.PackageTypeName AS Package,
                    IL.Quantity,
                    IL.UnitPrice,
                    IL.TaxRate,
                    IL.ExtendedPrice - IL.TaxAmount AS TotalExcludingTax,
                    IL.TaxAmount,
                    IL.LineProfit AS Profit,
                    IL.ExtendedPrice AS TotalIncludingTax,
                    CASE 
                        WHEN SI.IsChillerStock = 0 THEN IL.Quantity 
                        ELSE 0 
                    END AS TotalDryItems,
                    CASE 
                        WHEN SI.IsChillerStock <> 0 THEN IL.Quantity 
                        ELSE 0 
                    END AS TotalChillerItems
                FROM 
                    Invoices I LEFT JOIN 
                    InvoiceLines IL ON 
                        I.InvoiceID = IL.InvoiceID LEFT JOIN 
                    [__StockItems_Available_{{new_cutoff}}] SI ON 
                        IL.StockItemID = SI.StockItemID LEFT JOIN 
                    [__PackageTypes_Available_{{new_cutoff}}] PT ON 
                        IL.PackageTypeID = PT.PackageTypeID LEFT JOIN 
                    [__Customers_Available_{{new_cutoff}}] C ON 
                        I.CustomerID = C.CustomerID 
                WHERE 
                    (
                        DATETIME(I.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        DATETIME(IL.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            (
                                DATETIME(I.LastEditedWhen) >= DATETIME('{{new_cutoff}}') OR
                                DATETIME(IL.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                            )
                        ) OR
                        IL.PackageTypeID IN 
                        (
                            SELECT 
                                PTA.PackageTypeID 
                            FROM 
                                [__PackageTypes_Available_{{new_cutoff}}] PTA
                        )
                    ) AND
                    (
                        DATETIME(I.LastEditedWhen) <= DATETIME('{{new_cutoff}}') OR
                        DATETIME(IL.LastEditedWhen) <= DATETIME('{{new_cutoff}}') 
                    )
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ).replace(
               "{{last_cutoff}}", 
                last_cutoff
            ).replace(
               "{{initial_load}}", 
                initial_load
            ),
            sql_archive_where="", 
            table_name="Invoices", 
            archive_table_name="", 
            save_table_name="__FctSaleTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 5: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctSale
                    (
                        SaleKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        CityKey INTEGER,
                        CustomerKey INTEGER,
                        BillToCustomerKey INTEGER,
                        StockItemKey INTEGER,
                        InvoiceDateKey TEXT,
                        DeliveryDateKey TEXT,
                        SalesPersonKey INTEGER,
                        WWIInvoiceID INTEGER,
                        WWIInvoiceLineID INTEGER,
                        Description TEXT,
                        Package TEXT,
                        Quantity INTEGER,
                        UnitPrice REAL,
                        TaxRate REAL,
                        TotalExcludingTax REAL,
                        TaxAmount REAL,
                        Profit REAL,
                        TotalIncludingTax REAL,
                        TotalDryItems INTEGER,
                        TotalChillerItems INTEGER
                    )
                """,
                # Update CityKey
                """
                    UPDATE
                        __FctSaleTemp
                    SET
                        CityKey = DimCity.CityKey
                    FROM
                        DimCity
                    WHERE
                        DimCity.WWICityID = __FctSaleTemp.WWICityID
                """,
                # Update CustomerKey
                """
                    UPDATE
                        __FctSaleTemp
                    SET
                        CustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctSaleTemp.WWICustomerID
                """,
                # Update BillToCustomerKey
                """
                    UPDATE
                        __FctSaleTemp
                    SET
                        BillToCustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctSaleTemp.WWIBillToCustomerID
                """,
                # Update StockItemKey
                """
                    UPDATE
                        __FctSaleTemp
                    SET
                        StockItemKey = DimStockItem.StockItemKey
                    FROM
                        DimStockItem
                    WHERE
                        DimStockItem.WWIStockItemID = __FctSaleTemp.WWIStockItemID
                """,
                # Update SalesPersonKey
                """
                    UPDATE
                        __FctSaleTemp
                    SET
                        SalesPersonKey = DimEmployee.EmployeeKey
                    FROM
                        DimEmployee
                    WHERE
                        DimEmployee.WWIEmployeeID = __FctSaleTemp.WWISalesPersonID
                """,
                # Update existing data
                """
                    UPDATE
                        FctSale
                    SET
                        CityKey = __FctSaleTemp.CityKey,
                        CustomerKey = __FctSaleTemp.CustomerKey,
                        BillToCustomerKey = __FctSaleTemp.BillToCustomerKey,
                        StockItemKey = __FctSaleTemp.StockItemKey,
                        InvoiceDateKey = SUBSTR(__FctSaleTemp.InvoiceDateKey, 1, 10),
                        DeliveryDateKey = SUBSTR(__FctSaleTemp.DeliveryDateKey, 1, 10),
                        SalesPersonKey = __FctSaleTemp.SalesPersonKey,
                        Description = __FctSaleTemp.Description,
                        Package = __FctSaleTemp.Package,
                        Quantity = __FctSaleTemp.Quantity,
                        UnitPrice = __FctSaleTemp.UnitPrice,
                        TaxRate = __FctSaleTemp.TaxRate,
                        TotalExcludingTax = __FctSaleTemp.TotalExcludingTax,
                        TaxAmount = __FctSaleTemp.TaxAmount,
                        Profit = __FctSaleTemp.Profit,
                        TotalIncludingTax = __FctSaleTemp.TotalIncludingTax,
                        TotalDryItems = __FctSaleTemp.TotalDryItems,
                        TotalChillerItems = __FctSaleTemp.TotalChillerItems
                    FROM
                        __FctSaleTemp
                    WHERE
                        FctSale.WWIInvoiceID = __FctSaleTemp.WWIInvoiceID AND
                        FctSale.WWIInvoiceLineID = __FctSaleTemp.WWIInvoiceLineID
                """,
                # Add missing data
                """
                    INSERT INTO FctSale
                    (
                        CityKey,
                        CustomerKey,
                        BillToCustomerKey,
                        StockItemKey,
                        InvoiceDateKey,
                        DeliveryDateKey,
                        SalesPersonKey,
                        WWIInvoiceID,
                        WWIInvoiceLineID,
                        Description,
                        Package,
                        Quantity,
                        UnitPrice,
                        TaxRate,
                        TotalExcludingTax,
                        TaxAmount,
                        Profit,
                        TotalIncludingTax,
                        TotalDryItems,
                        TotalChillerItems
                    )
                    SELECT 
                        FST.CityKey,
                        FST.CustomerKey,
                        FST.BillToCustomerKey,
                        FST.StockItemKey,
                        SUBSTR(FST.InvoiceDateKey, 1, 10),
                        SUBSTR(FST.DeliveryDateKey, 1, 10),
                        FST.SalesPersonKey,
                        FST.WWIInvoiceID,
                        FST.WWIInvoiceLineID,
                        FST.Description,
                        FST.Package,
                        FST.Quantity,
                        FST.UnitPrice,
                        FST.TaxRate,
                        FST.TotalExcludingTax,
                        FST.TaxAmount,
                        FST.Profit,
                        FST.TotalIncludingTax,
                        FST.TotalDryItems,
                        FST.TotalChillerItems
                    FROM 
                        __FctSaleTemp FST LEFT JOIN 
                        FctSale FS ON
                            FS.WWIInvoiceID  = FST.WWIInvoiceID AND
                            FS.WWIInvoiceLineID  = FST.WWIInvoiceLineID
                    WHERE
                        FS.WWIInvoiceID  IS NULL
                """
            ],
            db_wh
        )

        # Step 6: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctSale processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("FctSale processing failed...", e)
    finally:
        print("Completed processing FctSale...")

populate_fctsale()

Table exists __PackageTypes_Changed_2025-01-01
Table exists __PackageTypes_Available_2025-01-01
Table exists __StockItems_Available_2025-01-01


Table exists __Customers_Available_2025-01-01


Successfully created __FctSaleTemp table.


FctSale processing successful...
Completed processing FctSale...


### Populate FctStockHolding table

In [298]:
table_name = "FctStockHolding"

def populate_fctstockholding():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)

        # Step 2: Save temp table
        save_temp_table(
            sql_template="""
                SELECT  
                    NULL AS StockItemKey,
                    SIH.StockItemID AS WWIStockItemID,
                    SIH.QuantityOnHand,
                    SIH.BinLocation,
                    SIH.LastStocktakeQuantity,
                    SIH.LastCostPrice,
                    SIH.ReorderLevel,
                    SIH.TargetStockLevel
                FROM 
                    StockItemHoldings SIH
            """,
            sql_archive_where="", 
            table_name="StockItemHoldings", 
            archive_table_name="", 
            save_table_name="__FctStockHoldingTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 3: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctStockHolding
                    (
                        StockHoldingKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        StockItemKey INTEGER,
                        WWIStockItemID INTEGER,
                        QuantityOnHand INTEGER,
                        BinLocation TEXT,
                        LastStocktakeQuantity INTEGER,
                        LastCostPrice REAL,
                        ReorderLevel INTEGER,
                        TargetStockLevel INTEGER
                    )
                """,
                # Update StockItemKey
                """
                    UPDATE
                        __FctStockHoldingTemp
                    SET
                        StockItemKey = DimStockItem.StockItemKey
                    FROM
                        DimStockItem
                    WHERE
                        DimStockItem.WWIStockItemID = __FctStockHoldingTemp.WWIStockItemID
                """,
                # Update existing data
                """
                    UPDATE 
                        FctStockHolding
                    SET 
                        StockItemKey = __FctStockHoldingTemp.StockItemKey,
                        QuantityOnHand = __FctStockHoldingTemp.QuantityOnHand,
                        BinLocation = __FctStockHoldingTemp.BinLocation,
                        LastStocktakeQuantity = __FctStockHoldingTemp.LastStocktakeQuantity,
                        LastCostPrice = __FctStockHoldingTemp.LastCostPrice,
                        ReorderLevel = __FctStockHoldingTemp.ReorderLevel,
                        TargetStockLevel = __FctStockHoldingTemp.TargetStockLevel
                    FROM
                        __FctStockHoldingTemp
                    WHERE
                        FctStockHolding.WWIStockItemID = __FctStockHoldingTemp.WWIStockItemID 
                """,
                # Add missing data
                """
                    INSERT INTO FctStockHolding
                    (
                        StockItemKey,
                        WWIStockItemID,
                        QuantityOnHand,
                        BinLocation,
                        LastStocktakeQuantity,
                        LastCostPrice,
                        ReorderLevel,
                        TargetStockLevel
                    )
                    SELECT 
                        FSHT.StockItemKey,
                        FSHT.WWIStockItemID,
                        FSHT.QuantityOnHand,
                        FSHT.BinLocation,
                        FSHT.LastStocktakeQuantity,
                        FSHT.LastCostPrice,
                        FSHT.ReorderLevel,
                        FSHT.TargetStockLevel
                    FROM 
                        __FctStockHoldingTemp FSHT LEFT JOIN 
                        FctStockHolding FSH ON
                            FSH.WWIStockItemID  = FSHT.WWIStockItemID JOIN
                        DimStockItem DSI ON
                            DSI.StockItemKey = FSHT.StockItemKey
                    WHERE
                        FSH.WWIStockItemID  IS NULL
                """
            ],
            db_wh
        )

        # Step 4: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctStockHolding processing successful...")
    except e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print(e)
        print("FctStockHolding processing failed...")
    finally:
        print("Completed processing FctStockHolding...")

populate_fctstockholding()

Successfully created __FctStockHoldingTemp table.
FctStockHolding processing successful...
Completed processing FctStockHolding...


### Populate FctTransaction table

In [299]:
table_name = "FctTransaction"

def populate_fcttransaction():
    if table_loaded_for_date(table_name, new_cutoff):
        print(table_name + " is already loaded for " + new_cutoff + ".")
        return

    try:
        # STEP 1: Add entry in LoadHistory table
        insert_load_history_entry(table_name, new_cutoff)

        # Step 2: Save temp table
        save_temp_table(
            sql_template="""
                SELECT 
                    CT.TransactionDate AS DateKey,
                    NULL AS CustomerKey,
                    NULL AS BillToCustomerKey,
                    NULL AS SupplierKey,
                    NULL AS TransactionTypeKey,
                    NULL AS PaymentMethodKey,
                    COALESCE(I.CustomerID, CT.CustomerID) AS WWICustomerID,
                    CT.CustomerID AS WWIBillToCustomerID,
                    NULL AS WWISupplierID,
                    CT.TransactionTypeID AS WWITransactionTypeID,
                    CT.PaymentMethodID AS WWIPaymentMethodID,
                    CT.CustomerTransactionID AS WWICustomerTransactionID,
                    NULL AS WWISupplierTransactionID,
                    CT.InvoiceID AS WWIInvoiceID,
                    NULL AS WWIPurchaseOrderID,
                    NULL AS SupplierInvoiceNumber,
                    CT.AmountExcludingTax AS TotalExcludingTax,
                    CT.TaxAmount,
                    CT.TransactionAmount AS TotalIncludingTax,
                    CT.OutstandingBalance,
                    CT.IsFinalized
                FROM 
                    CustomerTransactions CT LEFT JOIN 
                    Invoices I ON 
                        CT.InvoiceID = I.InvoiceID
                WHERE 
                    (
                        DATETIME(CT.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(CT.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                        )
                    ) AND
                    DATETIME(CT.LastEditedWhen) <= DATETIME('{{new_cutoff}}')
                    
                UNION ALL

                SELECT 
                    ST.TransactionDate AS DateKey,
                    NULL AS CustomerKey,
                    NULL AS BillToCustomerKey,
                    NULL AS SupplierKey,
                    NULL AS TransactionTypeKey,
                    NULL AS PaymentMethodKey,
                    NULL AS WWICustomerID,
                    NULL AS WWIBillToCustomerID,
                    ST.SupplierID AS WWISupplierID,
                    ST.TransactionTypeID AS WWITransactionTypeID,
                    ST.PaymentMethodID AS WWIPaymentMethodID,
                    NULL AS WWICustomerTransactionID,
                    ST.SupplierTransactionID AS WWISupplierTransactionID,
                    NULL AS WWIInvoiceID,
                    ST.PurchaseOrderID AS WWIPurchaseOrderID,
                    ST.SupplierInvoiceNumber AS SupplierInvoiceNumber,
                    ST.AmountExcludingTax AS TotalExcludingTax,
                    ST.TaxAmount,
                    ST.TransactionAmount AS TotalIncludingTax,
                    ST.OutstandingBalance,
                    ST.IsFinalized 
                FROM 
                    SupplierTransactions ST
                WHERE 
                    (
                        DATETIME(ST.LastEditedWhen) > DATETIME('{{last_cutoff}}') OR
                        (
                            DATETIME('{{new_cutoff}}') = DATETIME('{{initial_load}}') AND
                            DATETIME(ST.LastEditedWhen) >= DATETIME('{{new_cutoff}}')
                        )
                    ) AND
                    DATETIME(ST.LastEditedWhen) <= DATETIME('{{new_cutoff}}')
            """.replace(
               "{{new_cutoff}}", 
                new_cutoff
            ).replace(
               "{{last_cutoff}}", 
                last_cutoff
            ).replace(
               "{{initial_load}}", 
                initial_load
            ),
            sql_archive_where="", 
            table_name="CustomerTransactions", 
            archive_table_name="", 
            save_table_name="__FctTransactionTemp", 
            source_url=url_db,
            destination_url=url_db_wh
        )


        # Step 3: Save DimCustomer Table
        sql_execute_scripts(
            [
                # Create the warehouse table if not exists
                """
                    CREATE TABLE IF NOT EXISTS FctTransaction
                    (
                        TransactionKey INTEGER PRIMARY KEY AUTOINCREMENT,
                        DateKey TEXT,
                        CustomerKey INTEGER,
                        BillToCustomerKey INTEGER,
                        SupplierKey INTEGER,
                        TransactionTypeKey INTEGER,
                        PaymentMethodKey INTEGER,
                        WWICustomerID INTEGER,
                        WWIBillToCustomerID INTEGER,
                        WWISupplierID INTEGER,
                        WWITransactionTypeID INTEGER,
                        WWIPaymentMethodID INTEGER,
                        WWICustomerTransactionID INTEGER,
                        WWISupplierTransactionID INTEGER,
                        WWIInvoiceID INTEGER,
                        WWIPurchaseOrderID INTEGER,
                        SupplierInvoiceNumber TEXT,
                        TotalExcludingTax REAL,
                        TaxAmount REAL,
                        TotalIncludingTax REAL,
                        OutstandingBalance REAL,
                        IsFinalized INTEGER
                    )
                """,
                # Update CustomerKey
                """
                    UPDATE
                        __FctTransactionTemp
                    SET
                        CustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctTransactionTemp.WWICustomerID
                """,
                # Update BillToCustomerKey
                """
                    UPDATE
                        __FctTransactionTemp
                    SET
                        BillToCustomerKey = DimCustomer.CustomerKey
                    FROM
                        DimCustomer
                    WHERE
                        DimCustomer.WWICustomerID = __FctTransactionTemp.WWIBillToCustomerID
                """,
                # Update SupplierKey
                """
                    UPDATE
                        __FctTransactionTemp
                    SET
                        SupplierKey = DimSupplier.SupplierKey
                    FROM
                        DimSupplier
                    WHERE
                        DimSupplier.WWISupplierID = __FctTransactionTemp.WWISupplierID
                """,
                # Update TransactionTypeKey
                """
                    UPDATE
                        __FctTransactionTemp
                    SET
                        TransactionTypeKey = DimTransactionType.TransactionTypeKey
                    FROM
                        DimTransactionType
                    WHERE
                        DimTransactionType.WWITransactionTypeID = __FctTransactionTemp.WWITransactionTypeID
                """,
                # Update PaymentMethodKey
                """
                    UPDATE
                        __FctTransactionTemp
                    SET
                        PaymentMethodKey = DimPaymentMethod.PaymentMethodKey
                    FROM
                        DimPaymentMethod
                    WHERE
                        DimPaymentMethod.WWIPaymentMethodID = __FctTransactionTemp.WWIPaymentMethodID
                """,
                # Update existing data
                """
                    UPDATE 
                        FctTransaction
                    SET 
                        DateKey = SUBSTR(__FctTransactionTemp.DateKey, 1, 10),
                        CustomerKey = __FctTransactionTemp.CustomerKey,
                        BillToCustomerKey = __FctTransactionTemp.BillToCustomerKey,
                        SupplierKey = __FctTransactionTemp.SupplierKey,
                        TransactionTypeKey = __FctTransactionTemp.TransactionTypeKey,
                        PaymentMethodKey = __FctTransactionTemp.PaymentMethodKey,
                        WWICustomerID = __FctTransactionTemp.WWICustomerID,
                        WWIBillToCustomerID = __FctTransactionTemp.WWIBillToCustomerID,
                        WWISupplierID = __FctTransactionTemp.WWISupplierID,
                        WWITransactionTypeID = __FctTransactionTemp.WWITransactionTypeID,
                        WWIPaymentMethodID = __FctTransactionTemp.WWIPaymentMethodID,
                        WWIInvoiceID = __FctTransactionTemp.WWIInvoiceID,
                        WWIPurchaseOrderID = __FctTransactionTemp.WWIPurchaseOrderID,
                        SupplierInvoiceNumber = __FctTransactionTemp.SupplierInvoiceNumber,
                        TotalExcludingTax = __FctTransactionTemp.TotalExcludingTax,
                        TaxAmount = __FctTransactionTemp.TaxAmount,
                        TotalIncludingTax = __FctTransactionTemp.TotalIncludingTax,
                        OutstandingBalance = __FctTransactionTemp.OutstandingBalance,
                        IsFinalized = __FctTransactionTemp.IsFinalized
                    FROM
                        __FctTransactionTemp
                    WHERE
                        FctTransaction.WWICustomerTransactionID IS __FctTransactionTemp.WWICustomerTransactionID AND
                        FctTransaction.WWISupplierTransactionID IS __FctTransactionTemp.WWISupplierTransactionID
                """,
                # Add missing data
                """
                    INSERT INTO FctTransaction
                    (
                        DateKey,
                        CustomerKey,
                        BillToCustomerKey,
                        SupplierKey,
                        TransactionTypeKey,
                        PaymentMethodKey,
                        WWICustomerID,
                        WWIBillToCustomerID,
                        WWISupplierID,
                        WWITransactionTypeID,
                        WWIPaymentMethodID,
                        WWICustomerTransactionID,
                        WWISupplierTransactionID,
                        WWIInvoiceID,
                        WWIPurchaseOrderID,
                        SupplierInvoiceNumber,
                        TotalExcludingTax,
                        TaxAmount,
                        TotalIncludingTax,
                        OutstandingBalance,
                        IsFinalized
                    )
                    SELECT 
                        SUBSTR(FTT.DateKey, 1, 10),
                        FTT.CustomerKey,
                        FTT.BillToCustomerKey,
                        FTT.SupplierKey,
                        FTT.TransactionTypeKey,
                        FTT.PaymentMethodKey,
                        FTT.WWICustomerID,
                        FTT.WWIBillToCustomerID,
                        FTT.WWISupplierID,
                        FTT.WWITransactionTypeID,
                        FTT.WWIPaymentMethodID,
                        FTT.WWICustomerTransactionID,
                        FTT.WWISupplierTransactionID,
                        FTT.WWIInvoiceID,
                        FTT.WWIPurchaseOrderID,
                        FTT.SupplierInvoiceNumber,
                        FTT.TotalExcludingTax,
                        FTT.TaxAmount,
                        FTT.TotalIncludingTax,
                        FTT.OutstandingBalance,
                        FTT.IsFinalized
                    FROM 
                        __FctTransactionTemp FTT LEFT JOIN 
                        FctTransaction FT ON
                            FT.WWICustomerTransactionID IS FTT.WWICustomerTransactionID AND
                            FT.WWISupplierTransactionID IS FTT.WWISupplierTransactionID
                    WHERE
                        FT.WWICustomerTransactionID IS NULL AND
                        FT.WWISupplierTransactionID IS NULL 
                """
            ],
            db_wh
        )

        # Step 4: Update temp table status
        update_load_history_entry_status(table_name, new_cutoff, "Successful")
        print("FctTransaction processing successful...")
    except Exception as e:
        update_load_history_entry_status(table_name, new_cutoff, "Failed")
        print("FctTransaction processing failed...", e)
    finally:
        print("Completed processing FctTransaction...")

populate_fcttransaction()

Successfully created __FctTransactionTemp table.


FctTransaction processing successful...
Completed processing FctTransaction...


### Delete temp tables

In [300]:
def delete_temp_tables(source_url, source_db):
    # retrieve temp tables
    df = ps.read_sql("""
        SELECT 
            name
        FROM 
            sqlite_master 
        WHERE 
            type='table' AND
            name LIKE '/_/_%'  ESCAPE '/' 
        """,
        con=source_url
    )

    if len(df) > 0:
        for temp_table_name in df["name"].tolist():
            sql_execute_scripts(
                ["DROP TABLE [{{temp_table_name}}]".replace(
                    "{{temp_table_name}}", 
                    temp_table_name
                )], 
                source_db
            )
            print(temp_table_name + " dropped successfully in " + source_db + "...")
    else:
        print("No temp tables to delete in " + source_db + ".")

# Delete temp tables in source
delete_temp_tables(url_db, db)

# Delete temp tables in warehouse
delete_temp_tables(url_db_wh, db_wh)

__Countries_Changed_2025-01-01 dropped successfully in wwi.db...
__StateProvinces_Changed_2025-01-01 dropped successfully in wwi.db...
__Countries_Available_2025-01-01 dropped successfully in wwi.db...
__StateProvinces_Available_2025-01-01 dropped successfully in wwi.db...
__DimCityTemp dropped successfully in wwi.db...
__CustomerCategories_Changed_2025-01-01 dropped successfully in wwi.db...
__BuyingGroups_Changed_2025-01-01 dropped successfully in wwi.db...
__PrimaryContacts_Changed_2025-01-01 dropped successfully in wwi.db...
__CustomerCategories_Available_2025-01-01 dropped successfully in wwi.db...
__BuyingGroups_Available_2025-01-01 dropped successfully in wwi.db...
__PrimaryContacts_Available_2025-01-01 dropped successfully in wwi.db...
__Customers_Available_2025-01-01 dropped successfully in wwi.db...
__DimCustomerTemp dropped successfully in wwi.db...


__Employees_Changed_2025-01-01 dropped successfully in wwi.db...
__DimEmployeeTemp dropped successfully in wwi.db...
__PaymentMethods_Changed_2025-01-01 dropped successfully in wwi.db...
__DimPaymentMethodTemp dropped successfully in wwi.db...
__PackageTypes_Changed_2025-01-01 dropped successfully in wwi.db...
__Colors_Changed_2025-01-01 dropped successfully in wwi.db...
__PackageTypes_Available_2025-01-01 dropped successfully in wwi.db...
__Colors_Available_2025-01-01 dropped successfully in wwi.db...
__DimStockItemTemp dropped successfully in wwi.db...
__SupplierCategories_Changed_2025-01-01 dropped successfully in wwi.db...
__SupplierCategories_Available_2025-01-01 dropped successfully in wwi.db...
__DimSupplierTemp dropped successfully in wwi.db...
__TransactionTypes_Changed_2025-01-01 dropped successfully in wwi.db...
__DimTransactionTypeTemp dropped successfully in wwi.db...
__FctMovementTemp dropped successfully in wwi.db...


__FctOrderTemp dropped successfully in wwi.db...
__StockItems_Available_2025-01-01 dropped successfully in wwi.db...
__FctPurchaseTemp dropped successfully in wwi.db...
__FctSaleTemp dropped successfully in wwi.db...
__FctStockHoldingTemp dropped successfully in wwi.db...
__FctTransactionTemp dropped successfully in wwi.db...


__DimCityTemp dropped successfully in wwi_wh.db...
__DimCustomerTemp dropped successfully in wwi_wh.db...
__DimEmployeeTemp dropped successfully in wwi_wh.db...
__DimPaymentMethodTemp dropped successfully in wwi_wh.db...
__DimStockItemTemp dropped successfully in wwi_wh.db...
__DimSupplierTemp dropped successfully in wwi_wh.db...
__DimTransactionTypeTemp dropped successfully in wwi_wh.db...
__FctMovementTemp dropped successfully in wwi_wh.db...
__FctOrderTemp dropped successfully in wwi_wh.db...


__FctPurchaseTemp dropped successfully in wwi_wh.db...
__FctSaleTemp dropped successfully in wwi_wh.db...
__FctStockHoldingTemp dropped successfully in wwi_wh.db...
__FctTransactionTemp dropped successfully in wwi_wh.db...
